<a href="https://colab.research.google.com/github/Fatis092/repo52/blob/main/cv_car.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'cv-car-data:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F4115709%2F7133129%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240903%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240903T104035Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D7614216a59c0ead31e31598a8c4bd40b79ff6d53da731d6320e3794c546c9637d180d25fcc759792cba31b87cfebeab84da823073279ad14e7b68b526f4aea520764adb107a2142fa50de9a2a63d31165ad3c9f6504d18e02f6a5f64c2e3a36a5407a24a12d78e90301fa8045c92c98365be0cb09da7fdca3ec13251d733f1ff7c0b67e701e91c5623d478d9a4c85ff76499b1201f541180bcea5eddc99235107cf0e343605d80fcfa56068f335ce208f4e6a8bac6e1f4d2376af764ea30c5203ef831a19b1a766d2f327695f68321ce5e0a20c2b0fb872d5c2dec23045693f44082aeccf0a4560c6e23ab1f1f7f69e5cf1241f1563ef07fb605b9038e7ea31c'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


#### The computer vision project uses machine learning, radar technology, and OCR to analyze Egyptian license plates 🚘. EasyOCR library is employed for Arabic OCR. The code processes images, detects plate edges, and uses OCR for text. The results are displayed visually, demonstrating the integration of computer vision and OCR for license plate analysis. 🚘🚘

## 1 - Import librarys necessary to done the project

In [ ]:
import os
from easyocr import Reader
import cv2
import matplotlib.pylab as plt

## 2 - Reading the data set file

In [ ]:
image_folder = 'C:/Users/alabdeen/Desktop/project cv/data3'

### 3 - This line prepares an OCR reader in the Arabic programming language using a central processing unit (CPU) and without printing details of internal operations.

In [ ]:
reader = Reader(['ar'], gpu=False, verbose=False)

In [ ]:
# Image upload and processing:

for filename in os.listdir(image_folder):
    if filename.endswith(".jpg"):
        image_path = os.path.join(image_folder, filename)
        car = cv2.imread(image_path)
        car = cv2.resize(car, (800, 600))

        # Image processing for edge detection
        """
        The image is converted to grayscale and smoothed using a Gaussian Blur filter.
        Canny technology is then used to detect the edges of the board.
        """
        gray = cv2.cvtColor(car, cv2.COLOR_BGR2GRAY)
        blur = cv2.GaussianBlur(gray, (5, 5), 0)
        edged = cv2.Canny(blur, 10, 200)

        # Identify the panel:
        """
        The Contours technique is used to identify edges, and the five largest edges are selected.
        The plate is defined by searching for approximate polygons after finding the edge.
        """
        cont, _ = cv2.findContours(edged, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
        cont = sorted(cont, key=cv2.contourArea, reverse=True)[:5]
        plate_cnt = None
        for c in cont:
            arc = cv2.arcLength(c, True)
            approx = cv2.approxPolyDP(c, 0.02 * arc, True)
            if len(approx) == 4:
                plate_cnt = approx
                break

        # Read text from panel using OCR:
        """
        If the plate is found, OCR is used to read the text.
        If no text is readable, a message indicating that the text is difficult to read is displayed on the panel.
        If the text is read successfully, it is displayed on the image with the panel selected.
        """
        if plate_cnt is not None:
            (x, y, w, h) = cv2.boundingRect(plate_cnt)
            x = max(0, x)
            y = max(0, y)
            plate_roi = gray[y:y + h, x:x + w]

            detection = reader.readtext(plate_roi)

            if len(detection) == 0:
                text = "Impossible to read the text from the license plate"
                cv2.putText(car, text, (20, 40), cv2.FONT_HERSHEY_SIMPLEX, 0.75, (0, 0, 255), 3)
            else:
                cv2.drawContours(car, [plate_cnt], -1, (255, 0, 0), 3)
                text = f"{detection[0][1]} {detection[0][2] * 100:.2f}%"
                cv2.putText(car, text, (x, y - 20), cv2.FONT_HERSHEY_SIMPLEX, 0.75, (0, 255, 0), 2)
                print(text)

        # Display the original image
        plt.figure(figsize=(12, 6))
        plt.subplot(1, 3, 1)
        plt.imshow(cv2.cvtColor(car, cv2.COLOR_BGR2RGB))
        plt.title("Image")

        # If the board is found, display a thumbnail of the board
        if plate_cnt is not None:
            plt.subplot(1, 3, 2)
            plt.imshow(cv2.cvtColor(plate_roi, cv2.COLOR_BGR2RGB))
            plt.title("License Plate")

            # Display the original image with the panel selected
            plt.subplot(1, 3, 3)
            plt.imshow(cv2.cvtColor(car, cv2.COLOR_BGR2RGB))
            plt.title("Image with Contours")

        plt.show()
